In [5]:
from config import headers
import requests
import pandas as pd
from pprint import pprint
import os

### Pull List of Countries Available From Netflix API

In [13]:
url = "https://unogs-unogs-v1.p.rapidapi.com/aaapi.cgi"

In [14]:
querystring = {"t":"lc","q":"available"}

In [20]:
country_response = requests.request("GET", url, headers=headers, params=querystring)

In [58]:
# We will need the second value of every individual list, since it has the initials of each country, which will be used to make the lastest release API calls
country_response_json = country_response.json()['ITEMS']
country_response_json

[['21',
  'ar',
  'Argentina ',
  '1780',
  '1807',
  '17',
  '5709',
  '2049',
  '3660',
  'ARS',
  '109',
  '149',
  '189'],
 ['23',
  'au',
  'Australia ',
  '1161',
  '1188',
  '24',
  '5867',
  '2058',
  '3809',
  'AUD',
  '8.99',
  '11.99',
  '14.99'],
 ['26',
  'be',
  'Belgium ',
  '443',
  '451',
  '26',
  '5335',
  '1823',
  '3512',
  'EUR',
  '7.99',
  '9.99',
  '11.99'],
 ['29',
  'br',
  'Brazil ',
  '11',
  '37',
  '3',
  '1904',
  '759',
  '1145',
  'BRL',
  '19.9',
  '22.9',
  '29.9'],
 ['33',
  'ca',
  'Canada ',
  '925',
  '952',
  '29',
  '6005',
  '1988',
  '4017',
  'CAD',
  '7.99',
  '9.99',
  '11.99'],
 ['307',
  'cz',
  'Czech Republic ',
  '62',
  '146',
  '29',
  '6198',
  '2006',
  '4192',
  'EUR',
  '7.99',
  '9.99',
  '11.99'],
 ['45',
  'fr',
  'France ',
  '280',
  '308',
  '24',
  '4950',
  '1803',
  '3147',
  'EUR',
  '7.99',
  '9.99',
  '11.99'],
 ['39',
  'de',
  'Germany ',
  '1874',
  '1937',
  '18',
  '5686',
  '2050',
  '3636',
  'EUR',
  '7.99',


In [76]:
num_list = []
country_list = []
country_name_list = []

for num, country in enumerate(country_response_json):
    num_list.append(num)
    country_list.append(country[1].upper())
    country_name_list.append(country[2])

In [77]:
countries_df = pd.DataFrame({"Country_In": country_list,
                            "Country_Name": country_name_list})

In [78]:
countries_df = countries_df.set_index('Country_In')
countries_df

,Country_Name
Country_In,
AR,Argentina
AU,Australia
BE,Belgium
BR,Brazil
CA,Canada
CZ,Czech Republic
FR,France
DE,Germany
GR,Greece


### Iterate through List of Countries and Make API Calls to Obtain Latest Release for Each Country

In [79]:
url = "https://unogs-unogs-v1.p.rapidapi.com/aaapi.cgi"

for i in country_list:
    querystring = {"q":f"get:new7:{i}","p":"1","t":"ns","st":"adv"}

{'q': 'get:new7:AR', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:AU', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:BE', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:BR', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:CA', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:CZ', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:FR', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:DE', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:GR', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:HK', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:HU', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:IN', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:IL', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:IT', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:JP', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:MY', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:MX', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get:new7:NL', 'p': '1', 't': 'ns', 'st': 'adv'}
{'q': 'get

In [71]:
latest_release_response = requests.request("GET", url, headers=headers, params=querystring)

In [72]:
latest_release_response.json()['ITEMS'][0]

{'netflixid': '81324416',
 'title': '40 Sticks',
 'image': 'https://occ-0-179-299.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABVHNGfrvnZrezdej1r8gTH_WdJDyEwMrHuFPyzqUfC7QqdeHop35KUMivKpW-3baSC8jCYtJ-YfB3UxFke0FUepQCA.jpg?r=726',
 'synopsis': 'When their prison bus crashes in a forest on a rainy night, a group of criminals finds themselves battling wild animals and a mysterious killer.<br><b>New on 2020-11-28</b>',
 'rating': '',
 'type': 'movie',
 'released': '2019',
 'runtime': '1h32m',
 'largeimage': 'https://occ-0-1068-92.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABYDh8m5eaDkQQFju_vLU2HDqdygUByexTBU4oJzN4CmQ7JVFrtMY9lX12wSRyh9bRsvosH0CFRfYTol2X67j-9SRDg4x.jpg?r=689',
 'unogsdate': '2020-11-28',
 'imdbid': '',
 'download': '0'}